<a href="https://colab.research.google.com/github/gbessardon/Land_Cover_comparison/blob/main/S2GLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running environment options

In [1]:
colab=1 # 1 running on google collab
drive=1 # 1 need to mount google drive

## S2GLC file location

In [2]:
fns2glc='/content/drive/MyDrive/S2GLCmap/S2GLC_Europe_2017_v1.2.tif'
S2GLCfolder=fns2glc.replace('/S2GLC_Europe_2017_v1.2.tif','')

## LUCAS file location

In [3]:
fnshp='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_polygons.shp'
fncsv='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_attributes.csv'

# Run on collab and drive 
set colab=1 and drive=1  in running enviroment options

## MOUNT google drive

In [4]:
if drive==1:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


## install geopandas and pyrpoj for collab

In [5]:
if colab==1:
  !pip install geopandas
  !pip install pyproj

     |████████████████████████████████| 1.0 MB 14.8 MB/s 
     |████████████████████████████████| 15.4 MB 50.6 MB/s 
     |████████████████████████████████| 6.3 MB 58.5 MB/s 


# import libraires

In [6]:
import os
from osgeo import gdal
from shapely import geometry
import geopandas as gpd
import numpy as np
import shutil

# Create functions

## create directory to store the segment before zipping

In [7]:
def create_sgement_dir(path=os.getcwd()):
    outputdir=os.path.join(path,'LUCAS_Segment')
    if not os.path.isdir(outputdir):
      os.mkdir(outputdir)
    return outputdir

# Main

## Open LUCAS shapefile in a geodatabase

In [8]:
LUCASgdf = gpd.read_file(fnshp)

## Create directory to store the segment before zipping

In [9]:
outputdir=create_sgement_dir(path=os.getcwd())

## Create S2GLC segments corresponding to the LUCAS segments

In [ ]:
for i,d in LUCASgdf.iterrows():
      lucasp=LUCASgdf['geometry'].loc[i]
      bounds=lucasp.bounds
      lucasid=LUCASgdf['POINT_ID'].loc[i]
      outfile=os.path.join(outputdir,str(lucasid)+'.tif')
      opt=gdal.WarpOptions(dstSRS="EPSG:4326",outputBounds=bounds,
                           targetAlignedPixels=True,xRes=0.000089,yRes=0.000089)
      gdal.Warp(outfile,fns2glc,options=opt)

## Save in a zip archive in google drive (prevents I/O errors)

In [ ]:
shutil.make_archive(os.path.join(S2GLCfolder,'LUCAS_Segment'), 'zip', outputdir)